In [1]:
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model
import numpy as np
import os
import torch
from concrete_scars_dataset import ConcreteScarsDataset
from concrete_scars_model import ConcreteScarsModel

API_KEY = os.environ['COMETKEY']
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
# experiment = Experiment(
#   api_key = API_KEY,
#   project_name = 'concrete-scars-detection',
#   workspace='my-projects'
# )

In [3]:
hyper_params = {
    'learning_rate': 0.001,
    'epochs': 10,
    'batch_size': 10
}

In [4]:
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

dataset = ConcreteScarsDataset(transform=ToTensor()) #default distribution 300-negative 700-positive
data_loader = DataLoader(dataset=dataset, batch_size=hyper_params['batch_size'], shuffle=True, num_workers=1)
steps = len(dataset)

model = ConcreteScarsModel().to(DEVICE)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=hyper_params['learning_rate'])

In [5]:
for epoch in range(hyper_params['epochs']):
    error = np.zeros(steps)

    for i, (images, masks) in enumerate(data_loader):
        images = images.to(DEVICE)
        masks = masks.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)

        loss.backward()
        optimizer.step()

        error[i] = loss.item()
        #if(i%10==0):
            #ToDo poprawnie wyliczyc step
        print('\rStep: ', (i)*hyper_params['batch_size'], '/', steps, ' epoch: ', epoch+1, end='\r', flush=True)
    print('Epoch ', epoch+1, ' error: ', np.mean(error))
    

Epoch  0  error:  0.002463959224987775


In [ ]:
from torchvision.transforms import Normalize

class Detransform():
  def __init__(self, mean, std):
    self.mean = mean
    self.std = std
  
  # PIL images loaded into dataset are normalized.
  # In order to display them correctly we need to un-normalize them first
  def un_normalize_image(self, image):
    un_normalize = Normalize(
        (-self.mean / self.std), (1.0 / self.std)
    )
    return un_normalize(image)
  
  # If 'ToTensor' transformation was applied then the PIL images have CHW format.
  # To show them using pyplot.imshow(), we need to change it to HWC with 
  # permute() function.
  def reshape(self, image):
    return image.permute(1,2,0)

  def __call__(self, image):
    return self.reshape(self.un_normalize_image(image))



In [ ]:
import matplotlib.pyplot as plt
dataiter = iter(data_loader)
image, mask = next(dataiter)

image = image.to(DEVICE)
mask = mask.to(DEVICE)
predict = model(image)
detransform = Detransform(0,1)
maskd = detransform(mask[0])



In [ ]:
print(predict[0])
with torch.no_grad():
    i = 5
    img_dim = image[i].permute(1, 2, 0)
    predict_dim = predict[i].permute(1,2,0)
    msk_dim = mask[i].permute(1,2,0)
    plt.imshow(img_dim.cpu())
    plt.show()
    plt.imshow(predict_dim.cpu())
    plt.show()
    plt.imshow(msk_dim.cpu()/255)
